In [ ]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

from utils.dataloader import get_dataloader

In [ ]:
'''
실행시키지 말 것 dataset zip파일 압축 푸는 코드
'''
# import zipfile
# DIV2K = zipfile.ZipFile('/home/lahj91/SR/DIV2K_valid_HR.zip')
# DIV2K.extractall('/home/lahj91/SR/DIV2K')
# DIV2K.close()

In [144]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./SR_training_datasets/BSDS200')#해당주소에서 data 이름 list로 추출
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("BSDS200_DataName.csv", index=False) #csv 파일로 저장

In [290]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_train_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_train_DataName.csv", index=False) #csv 파일로 저장

In [299]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('./DIV2K/DIV2K_valid_HR')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("DIV2K_valid_DataName.csv", index=False) #csv 파일로 저장

In [192]:
'''
실행시키지 말 것 data 이름 저장하는 코드
'''
# name = os.listdir('/home/lahj91/SR/SR_testing_datasets/BSDS100')
# df = pd.DataFrame(name, columns = ['name']) #열 이름 = name 으로 list name을 저장
# df.to_csv("valid_BSDS200.csv", index=False) #csv 파일로 저장

In [7]:
'''
부가적인 함수들
'''
def img2tensor(img, normalize=True):
    # handle gray image
    if len(img.shape)==2:
        img = img[:,:,np.newaxis]
    img = np.transpose(img, axes=(2,0,1)).astype(np.float32)
    if normalize:
        img = img / img.max()
    return torch.from_numpy(img)

def tensor2img(tensor):
    if len(tensor.shape)==2:
        img = tensor.cpu().numpy()
    elif len(tensor.shape)==3:
        img = np.transpose(tensor.cpu().numpy(), axes=(1,2,0))
    return img

def read_img_as_tensor(img_path, img_size=224, normalize=True):
    if isinstance(img_size, int):
        img_size = (img_size, img_size)
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img2tensor(img, normalize=normalize)
    return img

In [1]:
import os
import numpy as np
import random
import cv2
import pandas as pd

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import Normalize

from dataloader import get_dataloader

In [2]:
data_loader = get_dataloader(batch_size=16,
                                    setting='train', 
                                    augmentation=True,
                                    pin_memory=True,
                                    SR_mode = 2,
                                    data='DIV2K')

In [3]:
for items in data_loader:
    items = items
    break